In [1]:
!git clone https://github.com/fvarga94/C_bytecode_analysis

Cloning into 'C_bytecode_analysis'...
remote: Counting objects: 636, done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 636 (delta 255), reused 428 (delta 185), pack-reused 134
Receiving objects: 100% (636/636), 11.64 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (328/328), done.


In [0]:
import tensorflow as tf
import glob
import os
import numpy as np

In [0]:
#data preparation
class Data:
    def __init__(self,source="/C_bytecode_analysis/output", seq_len=8):
        self.seq_len=seq_len
        self.home=os.getenv("HOME")
        self.source=source
        self.data=[]
        self.keywords=["FOR","CALL","IF","ELSE","WHILE","DO","SWITCH","FUNCTION"]
        self.number_of_lines=0
        self.data_label_dict={}
        for x in self.keywords:
            self.data_label_dict[x]=[]
        offset=0
        f_count=0
        files=glob.glob(os.path.join(self.home+self.source,'*/*.labeled_addresses'))
        f_max=len(files)
        for filename in files:
            if f_count%10==0:
              print (f_count,"/",f_max)
            f_count+=1
            with open(filename) as f:
                lines = f.readlines()
                for line in lines:
                    line=line.strip().split("\t")
                    if len(line)==3:
                        func_name=line[0]
                        classes=line[1]
                        number_of_lines=int(line[2])
                        label_list=[]
                        offset=len(self.data)
                        #print (func_name, classes, number_of_lines)
                    else:
                        #if len(line)<2:
                        #    continue
                        #address = line[0]
                        bytecode = line[1].strip()
                        #instruction = line[2].strip()
                        #source_line = int(line[3])
                        #source_file = line[4]
                        labels = line[5:]
                        label_list.append(labels)
                        #print (address,bytecode,instruction,source_line,source_file,labels)
                        indexes= (str(bytecode))[2:4]
                        if len(bytecode)<=10:
                            indexes+=str(bytecode)[4:min(len(bytecode),10)]
                            indexes+="g"*(10-len(bytecode))
                        else:
                            indexes+=(str(bytecode))[4:10]
                        new=[]
                        for x in indexes:
                                pom=[0]*16
                                if x!="g":
                                    pom[int(x,16)]=1
                                new.extend(pom)
                        self.data.append(np.array(new))
                        number_of_lines-=1
                        if number_of_lines==0:
                            cur=[]
                            counter={}
                            start_dict={}
                            i=-1
                            for labels in label_list:
                                i+=1
                                for label in labels:
                                    if label not in cur:
                                        cur.append(label)
                                        counter[label]=1
                                        start_dict[label]=i
                                    else:
                                        counter[label]+=1
                            for label in counter.keys():
                                self.data_label_dict[label.split("_")[0]].append((start_dict[label]+offset,counter[label]))
            #break
        self.data=np.array(self.data)
        self.labels_with_length=[]
        for x in self.data_label_dict.keys():
            cl=self.keywords.index(x)
            for y in self.data_label_dict[x]:
                self.labels_with_length.append([cl,y[0],y[1]])
        self.labels_with_length=np.array(self.labels_with_length)
                
    def equalize_classes(self):
        max_cl=0
        print (len(self.labels_with_length))
        self.labels_with_length=self.labels_with_length.tolist()
        for x in self.data_label_dict.keys():
            if len(self.data_label_dict[x])>max_cl:
              max_cl=len(self.data_label_dict[x])
        for x in self.data_label_dict.keys():
            add_cl=int(max_cl/len(self.data_label_dict[x]))-1
            cl=self.keywords.index(x)
            for i in range(add_cl):
              for y in self.data_label_dict[x]:
                  self.labels_with_length.append([cl,y[0],y[1]])
        self.labels_with_length=np.array(self.labels_with_length)
        print (len(self.labels_with_length))
    
    def set_batch_size(self,size=50):
        self.size=size
    
    def make_batches(self):
        perm=np.random.permutation(len(self.labels_with_length))
        self.y_l_batches=[]
        pom=self.labels_with_length[perm]
        for i in range(int(len(perm)/self.size)):
            self.y_l_batches.append(pom[i*self.size:i*self.size+self.size])
        self.y_l_batches=np.array(self.y_l_batches)
        self.num_batches=len(self.y_l_batches)
        self.current_batch_index=0
        
    def get_batch(self):
        seq_len=self.seq_len
        y_l_batch=self.y_l_batches[self.current_batch_index]
        pom=[self.data[i[1]:i[1]+i[2]] for i in y_l_batch]
        for i in range(len(pom)):
            if len(pom[i])>=seq_len*2:
                #print (len(pom[i]))
                pom[i]=np.vstack((pom[i][:seq_len],pom[i][-seq_len:]))
                #print (len(pom[i]))
                #print ("----")
            else:
                pom[i]=np.vstack([pom[i],np.array([np.zeros(128) for j in range(seq_len*2-len(pom[i]))])])
        x=np.array(pom)
        pom=np.array(y_l_batch[:,0])
        y=np.eye(8)[pom]
        self.current_batch_index+=1
        if self.current_batch_index==self.num_batches:
            self.make_batches()
        return x,y

In [0]:
class ConvNet:
    def __init__(self,data):
        self.data=data
        
    @staticmethod
    def weight_variable(shape):
      initial = tf.truncated_normal(shape, stddev=0.1)
      return tf.Variable(initial)
    
    @staticmethod
    def bias_variable(shape):
      initial = tf.constant(0.1, shape=shape)
      return tf.Variable(initial)
    
    @staticmethod
    def conv2d(x, W):
      return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

    @staticmethod
    def max_pool_shape(x, shape=[1,1,2,1]):
      return tf.nn.max_pool(x, ksize=shape,
                            strides=shape, padding='VALID')

    def convolution_graph(self,conv_shape=[[5,5,1,16],[5,5,16,32]],fc_shape=[512,8], param_lambda=0.01, seq_len=8):
        
        self.x = tf.placeholder(tf.float32, shape=[None, seq_len*2, 128], name="x")
        self.y_ = tf.placeholder(tf.float32, shape=[None, 8], name="y_")
        self.W_conv=[]
        self.b_conv=[]
        self.h_conv=[]
        self.h_pool=[]
        
        global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = param_lambda
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.85, staircase=True)

        
        self.x_input= tf.reshape(self.x, [-1, 128, seq_len*2, 1])
        print (self.x_input.shape)
        self.layer_input=[]
        self.layer_input.append(self.x_input)
        #print (self.layer_input[-1].shape)
        
        for i in range(len(conv_shape)):
            self.W_conv.append(self.weight_variable(conv_shape[i]))
            self.b_conv.append(self.bias_variable([conv_shape[i][-1]]))

            self.h_conv.append(tf.nn.relu(self.conv2d(self.layer_input[-1], self.W_conv[-1]) + self.b_conv[-1]))
            self.layer_input.append(self.max_pool_shape(self.h_conv[-1]))
            #print (self.layer_input[-1].shape)
        
        fc_input_shape=[]
        a=1
        for b in self.layer_input[-1].shape[1:]:
            a*=int(b)
        fc_input_shape.append(a)
        
        self.W_fc=[]
        self.b_fc=[]
        self.h_pool_flat=[]
        
        
        for i in range(len(fc_shape[:-1])):
            
            self.W_fc.append(self.weight_variable([fc_input_shape[-1], fc_shape[i]]))
            self.b_fc.append(self.bias_variable([fc_shape[i]]))

            self.h_pool_flat.append(tf.reshape(self.layer_input[-1], [-1, fc_input_shape[-1]]))
            self.layer_input.append(tf.nn.relu(tf.matmul(self.h_pool_flat[-1], self.W_fc[-1]) + self.b_fc[-1]))
            
            fc_input_shape.append(fc_shape[i])
            #self.keep_prob = tf.placeholder(tf.float32)
            #self.h_fc1_drop = tf.nn.dropout(self.h_fc1, self.keep_prob)

        self.W_fc.append(self.weight_variable([fc_input_shape[-1], fc_shape[-1]]))
        self.b_fc.append(self.bias_variable([fc_shape[-1]]))
        self.y_conv = tf.matmul(self.layer_input[-1], self.W_fc[-1]) + self.b_fc[-1]

        self.l2=sum([param_lambda*tf.nn.l2_loss(x) for pom in [self.W_conv,self.W_fc] for x in pom])

        self.cross_entropy = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.y_conv))+self.l2
        self.train_step = tf.train.AdamOptimizer(learning_rate)
        
        self.learning_step = self.train_step.minimize(self.cross_entropy, global_step=global_step)

        
        correct_prediction = tf.equal(tf.argmax(self.y_conv, 1), tf.argmax(self.y_, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def train(self):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            self.data.make_batches()
            for i in range(10*self.data.num_batches):
                batch = self.data.get_batch()
                sess.run(self.learning_step,feed_dict={self.x: batch[0], self.y_: batch[1]})
                if i%self.data.num_batches==0:
                    acc= self.accuracy.eval(feed_dict={self.x: batch[0], self.y_: batch[1]})
                    print (acc, sess.run(self.train_step._lr))
                   

In [48]:
ext_data=Data()

0 / 399
10 / 399
20 / 399
30 / 399
40 / 399
50 / 399
60 / 399
70 / 399
80 / 399
90 / 399
100 / 399
110 / 399
120 / 399
130 / 399
140 / 399
150 / 399
160 / 399
170 / 399
180 / 399
190 / 399
200 / 399
210 / 399
220 / 399
230 / 399
240 / 399
250 / 399
260 / 399
270 / 399
280 / 399
290 / 399
300 / 399
310 / 399
320 / 399
330 / 399
340 / 399
350 / 399
360 / 399
370 / 399
380 / 399
390 / 399


In [0]:
convo_network=ConvNet(ext_data)

In [60]:
convo_network.data.equalize_classes()

725488
1027337


In [61]:
for x in convo_network.data.data_label_dict.keys():
    print (x,len(convo_network.data.data_label_dict[x]))
print (convo_network.data.data.shape)

FOR 3381
CALL 57134
IF 39936
ELSE 8921
WHILE 1943
DO 173
SWITCH 540
FUNCTION 9762
(881345, 128)


In [62]:
convo_network.convolution_graph(conv_shape=[[128,5,1,8]],fc_shape=[10,8], param_lambda=1e-4)
convo_network.data.set_batch_size(512)
convo_network.train()

(?, 128, 16, 1)
0.14257812 1e-04
0.7128906 7.225e-05
0.8222656 5.2200627e-05
0.8359375 3.771496e-05
0.8496094 2.7249056e-05
0.8535156 1.9687444e-05
0.8535156 1.4224181e-05
0.8671875 1.027697e-05
0.84375 7.4251116e-06
0.8613281 5.364643e-06
